In [27]:
#библиотеки, необходимые для формирования запросов сайту и получение ответов
#и поиска в полученной с сайта информации необхоимых для проекта данных
from bs4 import BeautifulSoup
import requests as req
import lxml.html as html
import urllib3

import time #библиотека для установки паузы между запросами к сайту

#общие библиотеки
import pandas as pd
import numpy as np

import re

In [28]:
#формируем header для включения его в запросы, так как сайты могут блокировать запросы без него
header = {'user-agent': 'Chrome/108.0.0.0'}
urllib3.disable_warnings()

In [32]:
url_star = 'https://www.booking.com/searchresults.html?label=gen173nr-1FCAEoggI46AdIM1gEaMIBiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuALQhLOcBsACAdICJDVhODVhNGQ4LTQ2ZDgtNGNkMi05YjgwLTA0OWU4NDY1ZGRmMNgCBeACAQ&sid=10047ad366873316955a6d37c8047d00&aid=304142&tmpl=searchresults&ac_click_type=b&ac_meta=GhBmMGYxNzA1MjlhMGEwMTNkIAAoATICZW46BU1pbGFuQABKAFAA&ac_position=0&class_interval=1&dest_id=-121726&dest_type=city&dtdisc=0&efdco=1&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&lang=en-us&nflt=ht_id%3D204%3Bclass%3D'
url_star_tail = '&no_rooms=1&postcard=0&raw_dest_type=city&room1=A%2CA&sb_price_type=total&sb_travel_purpose=leisure&search_selected=1&shw_aparth=1&slp_r_match=0&soz=1&src=searchresults&srpvid=c69f8e7aae9604e8&ss=Milan&ss_all=0&ssb=empty&sshis=0&ssne=Paris&ssne_untouched=Paris&lang_click=other&cdl=en-us&lang_changed=1&order=distance_from_search'
url_all_star = []

for i in range(2):
    url_all_star.append(url_star + str(i+4) + url_star_tail)

In [33]:
urls_hotel_4 = []
urls_hotel_5 = []

for star in range(2):
    time.sleep(3)   
    if star == 0:
        for i in range(0, 151, 25):
            response = req.get(url_all_star[star] + '&offset=' + str(i), headers=header, timeout=50, verify=False)
            page = BeautifulSoup(response.text,'html.parser')
            for a in page.find_all('a', class_='e13098a59f'):
                urls_hotel_4.append(a['href'])
            time.sleep(3) 
    if star == 1:
        for i in range(0, 26, 25):
            response = req.get(url_all_star[star] + '&offset=' + str(i), headers=header, timeout=50, verify=False)
            page = BeautifulSoup(response.text,'html.parser')
            for a in page.find_all('a', class_='e13098a59f'):
                urls_hotel_5.append(a['href'])
            time.sleep(3)
    
    

print(len(urls_hotel_4))
print(len(urls_hotel_5))

169
26


In [34]:
urls_hotel_4_en = []
urls_hotel_5_en = []


for i in range(len(urls_hotel_4)):
    parts = urls_hotel_4[i].split('hapos=' + str(i+1))
    urls_hotel_4_en.append(parts[0] + ('hapos=' + str(i+1)) + '&lang=en-us' + parts[1])
    
for i in range(len(urls_hotel_5)):
    parts = urls_hotel_5[i].split('hapos=' + str(i+1))
    urls_hotel_5_en.append(parts[0] + ('hapos=' + str(i+1)) + '&lang=en-us' + parts[1])


print(len(urls_hotel_4_en))
print(len(urls_hotel_5_en))

169
26


In [35]:
data_array_full = np.empty(4, 'str')

tags = ['span', 'div', 'div']
classes = ['hp_address_subtitle js-hp_address_subtitle jq_tooltip', 'd8eab2cf7f c90c0a70d3 db63693c62',
           'b5cd09854e f0d4d6a2f5 e46e88563a']

all_star_url = [urls_hotel_4_en, urls_hotel_5_en]


count_star_0 = 3

for urls in all_star_url:
    count_star_0 += 1
    for url in urls:
        try:
            response = req.get(url, headers=header, timeout=50, verify=False)
            page = BeautifulSoup(response.text,'html.parser')
        except:
            print('ошибка запроса')
        
    
        data = []
        data.append(count_star_0)
        for i in range(3):
            try:
                elem = page.find_all(tags[i], classes[i])
                elem = elem[0].text
            except:
                elem = np.NaN
        
                
            data.append(elem)
            
        data_array = np.asarray(data).reshape(1,4)
        data_array_full = np.row_stack((data_array_full,data_array))
    
        time.sleep(3)

ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса


In [62]:
columns = np.array(['hotel_star', 'hotel_address','reviews_count','quality'])
hotel_milan = pd.DataFrame(data=data_array_full, columns=columns)
hotel_milan.replace('nan', np.NaN, inplace=True)
hotel_milan = hotel_milan.dropna(thresh=2).reset_index(drop=True)
hotel_milan

,hotel_star,hotel_address,reviews_count,quality
0,,,,
1,4,"\nPiazza Duomo Angolo Via Mazzini , Milan City...",313 reviews,Wonderful
2,4,"\nVia Mazzini 4, Milan City Center, 20123 Mila...",987 reviews,Very Good
3,4,"\nSilvio Pellico 4, Milan City Center, 20121 M...","2,248 reviews",Wonderful
4,4,"\nVia Spadari, 1, Milan City Center, 20123 Mil...","1,871 reviews",Excellent
...,...,...,...,...
191,5,"\nVia Forcella 6, Navigli, 20144 Milan, Italy\n",674 reviews,Wonderful
192,5,"\nVia Emilio Cornalia 7, Central Station, 2012...",692 reviews,Very Good
193,5,"\n6 Via Aristotile Fioravanti, Garibaldi Stati...","2,052 reviews",Excellent
194,5,"\nPiazza Duca D'Aosta 9, Central Station, 2012...",573 reviews,Excellent


In [ ]:
dupl_columns_df = list(hotel_milan.columns) # создаем лист из колонок

# вычисляем количество дубликатов 
mask = hotel_milan.duplicated(subset=dupl_columns_df) 
hotel_duplicates_df = hotel_milan[mask]
print(f'Число найденных дубликатов: {hotel_duplicates_df.shape[0]}')

In [ ]:
hotel_milan = hotel_milan.drop_duplicates(subset=dupl_columns_df)
print(f'Результирующее число записей: {hotel_milan.shape[0]}')

In [71]:
hotel_df = pd.read_csv('C:/Users/Dmitriy/Desktop/SkillFactory/project_3/hotels.csv', sep=',')
hotel_df.shape

(386803, 57)

In [64]:
hotel_milan['hotel_address'] = hotel_milan['hotel_address'].apply(
    lambda x: re.sub("[^0-9a-zA-Z]"," ",x).lower().split(' ')[:4]).apply(
lambda x: ' '.join(x).replace(' ', ''))

In [73]:
hotel_df['hotel_address'] = hotel_df['hotel_address'].apply(
    lambda x: x.lower().split(' ')[:4]).apply(lambda x: ' '.join(x).replace(' ', ''))

In [75]:
hotel_df = hotel_df.merge(
                hotel_milan,
                on='hotel_address',
                how='left')

In [76]:
null = hotel_df[hotel_df['city_hotel'] == 'Milan'].isnull().mean() * 100
null_per = null[null > 0]
null_per

Unnamed: 0_y     18.205373
hotel_star       18.205373
reviews_count    18.205373
quality          18.205373
dtype: float64

In [1831]:
hotel_milan.to_csv('C:/Users/Dmitriy/Desktop/SkillFactory/project_3/parsing_hotels/hotel_milan.csv')